In [ ]:
from torchvision import models, transforms
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import pandas as pd
import os 
import math
import tqdm


image_dir = r'\Users\jqvil\Desktop\jessica quenneville\images\images'
csv_file = 'UTKFaceAugmented.csv'

data = pd.read_csv(csv_file)
# Placeholder to store image data and labels
images = []
labels = []

# Load and preprocess images
images = []
labels = []

for index, row in data.iterrows():
    filename = row['filename']
    age_label = row['age']
    image_path = os.path.join(image_dir, filename)
    
    if os.path.exists(image_path):
        img = Image.open(image_path).resize((64, 64)).convert('RGB')
        img_tensor = torch.tensor(np.array(img) / 255.0, dtype=torch.float32)
        images.append(img_tensor.unsqueeze(0))  # Add a batch dimension
        labels.append(age_label)
    else:
        print(f"File not found: {image_path}")

# After the loop, stack the list of tensors to create a single tensor
images_tensor = torch.cat(images, dim=0)
labels_tensor = torch.tensor(labels).float()

for filename in os.listdir(image_dir):
    # Check if the file is an image
    if filename.endswith(".jpg"):  # open files ending iwth .jpg
        # Construct the full path to the image file
        image_path = os.path.join(image_dir, filename)
        
        # Open the image using PIL
        img = Image.open(image_path)
        
        # Display the image 
        #img.show()

tensor_images = []

# Convert NumPy arrays to PyTorch tensors
for img in images:
    if isinstance(img, np.ndarray):
        img_tensor = torch.from_numpy(img)
        tensor_images.append(img_tensor)
    else:
        tensor_images.append(img)
# Concatenate tensors along the batch dimension (axis=0)
image_data = torch.cat(tensor_images, dim=0)

In [ ]:

# Load the tabular data
data = pd.read_csv('UTKFaceAugmented.csv')

features = data.drop(columns=['age']) # select age as column
labels = data['age']  # Replace with your age column

# Load a pre-trained image model (ResNet50 as an example)
image_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
image_model = nn.Sequential(*list(image_model.children())[:-1])  # Remove the last fully connected layer

# Freeze the image model's parameters
for param in image_model.parameters():
    param.requires_grad = False

In [ ]:
from dataloader import CustomMultimodalDataloader
from networks import MultiModalNet

In [ ]:
# Extract the necessary information for your model
num_tabular_features = len(features.columns)


train_dataloader = CustomMultimodalDataloader(images_tensor, labels_tensor, labels, batch_size=16)
# Define your loss function and 

model = MultiModalNet(image_model, num_tabular_features)
criterion = nn.MSELoss()  # Use appropriate loss function for regression task
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

# Training loop (you'll need to adapt this based on your dataset)


num_epochs = 50
batch_size = 32  # Adjust this according to your dataset

for epoch in tqdm.tqdm(range(num_epochs)):
    # Calculate the number of batches for this epoch
    num_batches = train_dataloader.num_batches_per_epoch // batch_size
    train_loss = []
    losses = []
    for batch_idx in range(num_batches):
        optimizer.zero_grad()

        # Fetch a batch from the DataLoader
        train_batch = train_dataloader.fetch_batch()
        yhat = model(train_batch['x_batch'])
        train_loss = torch.mean(optimizer(yhat.float(), train_batch['y_batch'].float()))

        # Perform forward pass, compute loss, backward pass, and optimize
        train_loss.backward()
        optimizer.step()
        losses.append(train_loss.detach().numpy())

    # personally, I like to visualize the loss per every iteration, rather than every epoch. I find it more useful to diagnose issues

        losses.append(train_loss.detach().numpy())
    train_loss.extend(losses)
    losses = []


In [ ]:

def predict_age(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    predicted_ages = []
    for _ in range(dataloader.num_batches_per_epoch):
        val_batch = dataloader.fetch_batch()
        x_batch, y_batch = val_batch['images'], val_batch['labels'] 
        yhat = model(x_batch)
        predicted_ages.extend(yhat.cpu().numpy()) 
        return predicted_ages

#predicted_ages = predict_age(model, train_dataloader)

